In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os


# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
dataset_path = "/home/kevin/datasets/netflix/"

In [5]:
read = dataset_path + "/README"
! cat $read

In [ ]:
! tail /kaggle/input/netflix-prize-data/combined_data_1.txt

In [44]:
# index by both movies and users
current_movie = 0
data_by_movie = {}

for i in range(1, 5):
    print(i)
    with open(dataset_path + '/combined_data_' + str(i) + '.txt') as f:
        for line in f:
            line = line.split(',')
            if len(line) == 1:
                current_movie = line[0][:-2]
                data_by_movie[current_movie] = {}
            else:
                current_user = line[0]
                data_by_movie[current_movie][current_user] = line[1:]
        
with open('data_by_movie_all.npy', 'wb') as f:
    np.save(f, data_by_movie)


1
2
3
4


In [43]:
data = np.load('data_by_movie1.npy', 'r')

ValueError: Array can't be memory-mapped: Python objects in dtype.

In [10]:
"""
with open('data_by_movie.npy', 'wb') as f:
    np.save(f, data_by_movie)
    
with open('data_by_user.npy', 'wb') as f:
    np.save(f, data_by_user)

len(data_by_movie.keys())

len(data_by_user.keys())

data_by_movie['1']

ratings = []
for movie in data_by_movie.keys():
    for user in data_by_movie[movie].keys():
        ratings.append(data_by_movie[movie][user][0])

average_rating = np.average(np.array([int(x) for x in ratings]))
"""

"\nwith open('data_by_movie.npy', 'wb') as f:\n    np.save(f, data_by_movie)\n    \nwith open('data_by_user.npy', 'wb') as f:\n    np.save(f, data_by_user)\n\nlen(data_by_movie.keys())\n\nlen(data_by_user.keys())\n\ndata_by_movie['1']\n\nratings = []\nfor movie in data_by_movie.keys():\n    for user in data_by_movie[movie].keys():\n        ratings.append(data_by_movie[movie][user][0])\n\naverage_rating = np.average(np.array([int(x) for x in ratings]))\n"

In [38]:
import numpy.linalg as LA

latent_dim = 5
"""
Notes
randomly initialize q and p, or initialize as zeros initially
"""
def min_function(r, q, p, lamb):
    count = 0
    total = 0
    for movie in r.keys():
        for user in r[movie].keys():
            count += 1
            rating = float(r[movie][user][0])
            if movie not in q:
                q[movie] = np.random.rand(latent_dim,1)
            if user not in p:
                p[user] = np.random.rand(latent_dim,1)
            prediction = np.dot(q[movie].T, p[user]) 
            norm_q_i = LA.norm(q[movie]) 
            norm_p_u = LA.norm(p[user])
            total += (rating-prediction)**2 + lamb*(norm_q_i + norm_p_u)
            if count % 10000 == 0:
                print("Count", count, "Total", total)
        if count >= 1000000:
            break

q = {}
p = {}
min_function(data_by_movie, q, p, 0.1)




Count 10000 Total [[59593.5490582]]
Count 20000 Total [[108700.60107372]]
Count 30000 Total [[158908.00455203]]
Count 40000 Total [[206774.06589883]]
Count 50000 Total [[260879.72382624]]
Count 60000 Total [[330506.82158083]]
Count 70000 Total [[404477.46730154]]
Count 80000 Total [[478327.86354923]]
Count 90000 Total [[552430.43961205]]
Count 100000 Total [[644005.1864228]]
Count 110000 Total [[742516.99364892]]
Count 120000 Total [[840478.9446101]]
Count 130000 Total [[939247.36110412]]
Count 140000 Total [[1038014.02077253]]
Count 150000 Total [[1136154.839783]]
Count 160000 Total [[1234119.93136704]]
Count 170000 Total [[1332063.32693328]]
Count 180000 Total [[1429880.22411054]]
Count 190000 Total [[1528323.4838531]]
Count 200000 Total [[1625849.60663246]]
Count 210000 Total [[1723374.75193003]]
Count 220000 Total [[1829083.30150738]]
Count 230000 Total [[1890949.07875622]]
Count 240000 Total [[1946497.79013118]]
Count 250000 Total [[2004552.61094575]]
Count 260000 Total [[2084736.

In [39]:
def sgd_function(r, q, p, lamb, lr):
    count = 0
    total = 0
    for movie in r.keys():
        for user in r[movie].keys():
            count += 1
            rating = float(r[movie][user][0])
            prediction = np.dot(q[movie].T, p[user]) 
            e_ui = rating-prediction
            updated_q_i = q[movie] + lr*(e_ui*p[user] - lamb*q[movie])
            updated_p_u = p[user] + lr*(e_ui*q[movie] - lamb*p[user])
            q[movie] = updated_q_i
            p[user] = updated_p_u
            # should SGD be all after? Let's do present for now
        if count >= 1000000:
            break
            
sgd_function(data_by_movie, q, p, 0.1, 0.1)

In [40]:
min_function(data_by_movie, q, p, 0.1)

Count 10000 Total [[21681.69218347]]
Count 20000 Total [[45162.51237943]]
Count 30000 Total [[64427.98064641]]
Count 40000 Total [[86049.7890805]]
Count 50000 Total [[103616.91421075]]
Count 60000 Total [[127318.89566651]]
Count 70000 Total [[154207.5667278]]
Count 80000 Total [[180856.0298366]]
Count 90000 Total [[207825.60882472]]
Count 100000 Total [[225292.04581417]]
Count 110000 Total [[240232.58494873]]
Count 120000 Total [[255775.30529466]]
Count 130000 Total [[270887.50738186]]
Count 140000 Total [[286211.2338547]]
Count 150000 Total [[301560.43654101]]
Count 160000 Total [[316757.86769721]]
Count 170000 Total [[331835.8738666]]
Count 180000 Total [[347038.52015384]]
Count 190000 Total [[362358.20022462]]
Count 200000 Total [[377438.83295516]]
Count 210000 Total [[392926.35897205]]
Count 220000 Total [[412065.45632751]]
Count 230000 Total [[428039.16747175]]
Count 240000 Total [[445232.63500109]]
Count 250000 Total [[463363.56350758]]
Count 260000 Total [[478830.78023237]]
Coun